# Introduction

I will be scraping homes.com site for rentals as I am interested in seeing how much the rental is costing now in the Nothern VA area. Northern Virginia is a suburban area outside of DC, so it is a good location especially for those who are working in Washington, DC. This area is crowded as there are lots of jobs here. Specially, I am looking for rentals in Fairfax County, Arlington County, and Washington DC. The search is restricted to townhouses and houses which have 2 or more bedrooms as I am more interested to see what the average rental price for the location given the rocket rising rental market.

I am going to scrape homes.com using Beautiful Soup to extract posts of title, price, address, the number of bedrooms, the number of bathrooms for the data analysis.

# Questions

I like to start off with questions for my project: 

- How much is the price range for 2-3 bedroom townhomes or family house
in Arlington County? Fairfax County? or DC?
- Is there a correlation between price vs the number of bathrooms? Location? or the number bedrooms?

# Webscraping

As a good practice for webscraping, firstly I was checking robots.txt file of homes.com site and see that I should 
be able to scrape the site as long as I leave out the following domains: /services/ and /routes as below:




Below is the result of homes.com/robots.txt: we are allowed to scrape as long as not following paths like /services/ or /routes/  

User-agent: *  
User-agent: AdsBot-Google  
User-agent: AdsBot-Google-Mobile  
Disallow: /services/  
Disallow: /routes/  

User-agent: GPTBot  
Disallow: /  

Sitemap: https://www.homes.com/sitemap-cities-for-sale-index.xml.gz  
Sitemap: https://www.homes.com/sitemap-cities-for-rent-index.xml.gz  
Sitemap: https://www.homes.com/sitemap-cities-openhouse-index.xml.gz  
Sitemap: https://www.homes.com/sitemap-counties-for-sale-index.xml.gz  
Sitemap: https://www.homes.com/sitemap-counties-for-rent-index.xml.gz  
Sitemap: https://www.homes.com/sitemap-zipcodes-for-sale-index.xml.gz  
Sitemap: https://www.homes.com/sitemap-zipcodes-for-rent-index.xml.gz  
Sitemap: https://www.homes.com/sitemap-neighborhoods-index.xml.gz  
Sitemap: https://www.homes.com/sitemap-neighborhoods-for-sale-index.xml.gz  
Sitemap: https://www.homes.com/sitemap-neighborhoods-for-rent-index.xml.gz  
Sitemap: https://www.homes.com/sitemap-agent-detail-index.xml.gz  
Sitemap: https://www.homes.com/sitemap-agent-directory-cities-index.xml.gz  
Sitemap: https://www.homes.com/sitemap-agent-directory-neighborhoods-index.xml.gz  
Sitemap: https://www.homes.com/sitemap-properties-active-index.xml.gz  
Sitemap: https://www.homes.com/sitemap-zipcodes-sold-index.xml.gz  
Sitemap: https://www.homes.com/sitemap-cities-sold-index.xml.gz  
Sitemap: https://www.homes.com/sitemap-schools-index.xml.gz  

# Imports

In [16]:
#!pip install plotly

In [17]:
#!pip install requests

In [18]:
#!pip install bs4

In [19]:
#!pip install pandas

In [2]:
#!pip install matplotlib

In [20]:
#!pip install seaborn

In [21]:
# keep all the required installed packages in requirements.txt
#!pip freeze > requirements.txt

In [2]:
# Importing necessary libraries
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
from time import sleep
import itertools
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import seaborn as sns
import re
import warnings
import numpy as np
import random
import plotly.express as px


I would tryout with one post to make sure the codes to extract price, address, bedroom, bathroom, and title are working as expected.
Trying out with the first post:
* I used requests and BeautifulSoup to download the content of the website
* Inspected the html structure and extracted the posts' details

In [14]:
# use a header to get the content 
Headers = ({'User-Agent': 
            'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.114 Safari/537.36'})

In [15]:
url = 'https://www.homes.com/fairfax-county-va/homes-for-rent/2-to-5-bedroom/p2/?property_type=1,2&bath-min=1&bath-max=5'
response = requests.get(url, headers=Headers)
first_soup = BeautifulSoup(response.text, "html.parser")

In [16]:
# checking the response status
response.status_code

200

In [1]:
# getting all posts on the first page
#posts = first_soup.find_all('li', class_= "placard-container")
#Trying out with post two
#post_two = posts[1]
#post_two
#print(posts)

Post Two

In [19]:
# Find all <li> elements within the <ul> element, we will get price, br, ba
li_elements = post_two.find('ul', class_='detailed-info-container').find_all('li')

# Extract the text from each <li> element
for li in li_elements:
    text = li.get_text()
    #print(type(text))
    print(text)

$3,400 per month
3 Beds
2.5 Baths


In [20]:
# extract addresses
address = post_two.find('p', class_='address').text
print(address)

5119 Bradfield Ct, Annandale, VA 22003


Post One

In [21]:
post_one= posts[0]
# Find all <li> elements within the <ul> element, we will get price, br, ba
li_elements = post_one.find('ul', class_='detailed-info-container').find_all('li')

# Extract the text from each <li> element
for li in li_elements:
    text = li.get_text()
    print(text)

$3,100 per month
3 Beds
3.5 Baths


In [22]:
# Initialize variables
price = None
bedrooms = None
bathrooms = None

# Find all <li> elements within the <ul> element
li_elements = post_one.find('ul', class_='detailed-info-container').find_all('li')

# Process the text from each <li> element
for li in li_elements:
    text = li.get_text()
    
    # Check if the text contains "per month" (indicating it's a price)
    if "per month" in text:
        if "-" in text:
            price = "Na"
        else:
            price = text[:-10]
            price = price.replace('$', '').replace(',', '')

# Now, outside the loop, check for "Beds" and "Baths"
for li in li_elements:
    text = li.get_text()
    
    # Check if the text contains "Beds" (indicating it's the number of bedrooms)
    if "Beds" in text:
        if "-" in text:
            bedrooms = "Na"
        else:
            bedrooms = int(text[:-5])

for li in li_elements:
    text = li.get_text()
    # Check if the text contains "Baths" (indicating it's the number of bathrooms)
    if "Baths" not in text:
        bathrooms = "Na"
    else:    
        bathrooms = float(text[:-6])

# Output the extracted data
print("Price:", price)
print("Bedrooms:", bedrooms)
print("Bathrooms:", bathrooms)


Price: 3100
Bedrooms: 3
Bathrooms: 3.5


In [23]:
# getting address
address = post_one.find('p', class_='address').text
print(address)

8914 Shamrock Ct, Springfield, VA 22152


In [24]:
# Define a regular expression pattern to capture city, state, and zipcode
pattern = r', (.*), (..) (\d{5})$'

# Use re.search() to find the pattern in the address
match = re.search(pattern, address)

# Check if a match is found and extract the city, state, and zipcode
if match:
    city = match.group(1).strip()  # Remove leading/trailing spaces
    state = match.group(2)
    zipcode = match.group(3)
    print("City:", city)
    print("State:", state)
    print("Zipcode:", zipcode)
else:
    print("No Address")


City: Springfield
State: VA
Zipcode: 22152


In [25]:
# getting titles
title = post_two.find('p', class_='property-name').text
print(title)

House for Rent


In [26]:
# extracting just Townhouse or House
pattern = r'(Townhouse|House) for Rent'
match = re.search(pattern, title)
if match:
    house_type = match.group(1)
    print(house_type)

House


Saving the html content

In [27]:
# save all the posts html content
# Convert the BeautifulSoup object back to a string
parsed_html = first_soup.prettify()  # You can use prettify() to make it nicely formatted

# Specify the file path where you want to save the parsed HTML content
file_path = 'parsed_file.html'

# Open the file in write mode and write the parsed HTML content to it
with open(file_path, 'w', encoding='utf-8') as file:
    file.write(parsed_html)

print(f'Parsed HTML content saved to {file_path}')


Parsed HTML content saved to parsed_file.html


In [28]:
# find total posts to use for pagination
total_posts = first_soup.find('span', attrs={'id':'spanSearchCount'}).text #spanSearchCount

In [29]:
total_posts

'518 2-Bedroom Homes for Rent in Fairfax County'

In [30]:
pattern = r'\d+'
match = re.search(pattern, total_posts)
total = match.group(0)
print(total)

518


## Combining all codes

After testing with post_two, I finally combined all the codes and added sleep command not to overload the site with several requests per second, so I would not get blocked.


# Fairfax County VA

In [31]:
#find the total number of pages
total_pages = (first_soup.find('span', class_='pageRange').text)
total_pages = int(total_pages[-2:])
print(total_pages)

21


In [33]:
# Define headers
Headers = ({'User-Agent': 
            'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.114 Safari/537.36'})

# Make a GET request to the initial page to get the total number of pages
response = requests.get("https://www.homes.com/fairfax-county-va/homes-for-rent/2-to-5-bedroom/?property_type=1,2&bath-min=1&bath-max=5", headers=Headers)

if response.status_code == 200:
    first_soup = BeautifulSoup(response.text, 'html.parser')
    # Find the total number of pages
    total_pages = first_soup.find('span', class_='pageRange').text
    total_pages = int(total_pages[-2:])
else:
    print("Failed to retrieve the total number of pages.")
    total_pages = 0

addresses = []
prices = []
bedrooms = []
bathrooms = []
titles = []

# Initiate pages
pages = np.arange(1, total_pages + 1)

i = 0

for page in pages:
    page_str = str(page)
    # Get request
    response = requests.get(f"https://www.homes.com/fairfax-county-va/homes-for-rent/2-to-5-bedroom/p{page_str}/?property_type=1,2&bath-min=1&bath-max=5", headers=Headers)
    
    # Pause the loop
    sleep(random.randint(5, 10))
    
    # Throw a warning for status codes that are not 200
    if response.status_code != 200:
        warnings.warn(f'Request for page {page} failed with status code: {response.status_code}')
        continue  # Skip processing this page and continue with the next
    
    # Define the HTML text
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Define the posts
    posts = soup.find_all('li', class_="placard-container")
    
    # Extract data item-wise
    for post in posts:
        if post.find('ul', class_='detailed-info-container') is not None:
            # Find all <li> elements within the <ul> element
            li_elements = post.find('ul', class_='detailed-info-container').find_all('li')

            # Initialize variables for price, bedroom, and bathroom
            price = "Na"
            bedroom = "Na"
            bathroom = "Na"

            # Process the text from each <li> element
            for li in li_elements:
                text = li.get_text()
                
                # Check if the text contains "per month" (indicating it's a price)
                if "per month" in text:
                    # Check if multiple prices (indicated by "-"), and assign "Na"
                    if "-" in text:
                        price = "Na"
                    else:
                        price = text[:-10]
                        # Otherwise, get only the price by removing "$" and ","
                        price = text.replace('$', '').replace(',', '')
                
                # Check if the text contains "Beds" (indicating it's the number of bedrooms)
                if "Beds" in text:
                    if "-" not in text:
                        bedroom = int(text.split()[0])

                # Check if the text contains "Baths" (indicating it's the number of bathrooms)
                if "Baths" in text:
                    if "-" not in text:
                        bathroom = float(text.split()[0])

            # Append the extracted data to respective lists
            prices.append(price)
            bedrooms.append(bedroom)
            bathrooms.append(bathroom)

        if post.find('p', class_='address') is not None:
            address = post.find('p', class_='address').text
            addresses.append(address)
        if post.find('p', class_='property-name') is not None:
            title = post.find('p', class_='property-name').text
            titles.append(title)
    
    i += 1
    print("Page " + str(i) + " scraped successfully!")

print("\n")
print("Scrape complete!")


Page 1 scraped successfully!
Page 2 scraped successfully!
Page 3 scraped successfully!
Page 4 scraped successfully!
Page 5 scraped successfully!
Page 6 scraped successfully!
Page 7 scraped successfully!
Page 8 scraped successfully!
Page 9 scraped successfully!
Page 10 scraped successfully!
Page 11 scraped successfully!
Page 12 scraped successfully!
Page 13 scraped successfully!
Page 14 scraped successfully!
Page 15 scraped successfully!
Page 16 scraped successfully!
Page 17 scraped successfully!
Page 18 scraped successfully!
Page 19 scraped successfully!
Page 20 scraped successfully!
Page 21 scraped successfully!


Scrape complete!


# Load and Store data

In [ ]:
titles[:20]

['Brentford at The Mile',
 'House for Rent',
 'Townes at Herndon',
 'Townhouse for Rent',
 'House for Rent',
 'Townhouse for Rent',
 'Townhouse for Rent',
 'House for Rent',
 'House for Rent',
 'Townhouse for Rent',
 'House for Rent',
 'House for Rent',
 'Townhouse for Rent',
 'Townhouse for Rent',
 'Townhouse for Rent',
 'House for Rent',
 'Townhouse for Rent',
 'Townhouse for Rent',
 'House for Rent',
 'House for Rent']

In [ ]:
addresses[:20]

['7970 Maitland St, McLean, VA 22102',
 '5929 Merritt Place, Falls Church, VA 22041',
 '508 Pride Ave, Herndon, VA 20170',
 'Unlisted Address',
 '703 Morningside Ct, Herndon, VA 20170',
 '5455 New London Park Dr, Fairfax, VA 22032',
 '14312 Beckett Glen Cir, Chantilly, VA 20151',
 '12665 Magna Carta Rd, Herndon, VA 20171',
 '1710 Palm Springs Dr, Vienna, VA 22182',
 '2013 Approach Ln, Reston, VA 20191',
 '7813 Mulberry Bottom Ln, Springfield, VA 22153',
 '5825 Biscayne Dr, Alexandria, VA 22303',
 '5415 Helm Ct, Fairfax, VA 22032',
 '2438 Ridgehampton Ct, Reston, VA 20191',
 '3160 Yeager Dr, Herndon, VA 20171',
 '3015 Fairmont St, Falls Church, VA 22042',
 '8046 Reserve Way, Vienna, VA 22182',
 '8043 Reserve Way, Vienna, VA 22182',
 '3506 Beverly Dr, Annandale, VA 22003',
 '855 Nicholas Run Dr, Great Falls, VA 22066']

In [ ]:
bedrooms[:10]

['Na', 5, 'Na', 3, 5, 3, 3, 3, 4, 3]

In [ ]:
bathrooms[:10]

['Na', 3.0, 'Na', 3.5, 2.0, 4.0, 5.0, 2.0, 2.5, 2.5]

In [ ]:
prices[:20]

['Na',
 '4150 per month',
 'Na',
 '2775 per month',
 '3300 per month',
 '2700 per month',
 '4800 per month',
 '2800 per month',
 '3750 per month',
 '2750 per month',
 '3800 per month',
 '3000 per month',
 '3200 per month',
 '2500 per month',
 '3500 per month',
 '3750 per month',
 '4000 per month',
 '4200 per month',
 '3800 per month',
 '14000 per month']

In [ ]:
# Create a data frame from extracted data above

import pandas as pd

fairfax_county_va = pd.DataFrame({'Title': titles,
                       'Address': addresses,
                       'Bedroom': bedrooms,
                       'Bathroom': bathrooms,
                        'Price': prices,
                        })
print(fairfax_county_va.info())
fairfax_county_va.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 504 entries, 0 to 503
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Title     504 non-null    object
 1   Address   504 non-null    object
 2   Bedroom   504 non-null    object
 3   Bathroom  504 non-null    object
 4   Price     504 non-null    object
dtypes: object(5)
memory usage: 19.8+ KB
None


,Title,Address,Bedroom,Bathroom,Price
0,Brentford at The Mile,"7970 Maitland St, McLean, VA 22102",Na,Na,Na
1,House for Rent,"5929 Merritt Place, Falls Church, VA 22041",5,3.0,4150 per month
2,Townes at Herndon,"508 Pride Ave, Herndon, VA 20170",Na,Na,Na
3,Townhouse for Rent,Unlisted Address,3,3.5,2775 per month
4,House for Rent,"703 Morningside Ct, Herndon, VA 20170",5,2.0,3300 per month
5,Townhouse for Rent,"5455 New London Park Dr, Fairfax, VA 22032",3,4.0,2700 per month
6,Townhouse for Rent,"14312 Beckett Glen Cir, Chantilly, VA 20151",3,5.0,4800 per month
7,House for Rent,"12665 Magna Carta Rd, Herndon, VA 20171",3,2.0,2800 per month
8,House for Rent,"1710 Palm Springs Dr, Vienna, VA 22182",4,2.5,3750 per month
9,Townhouse for Rent,"2013 Approach Ln, Reston, VA 20191",3,2.5,2750 per month


In [ ]:
#write to csv file for later use
fairfax_county_va.to_csv('fairfaxcountyVA.csv')

In [ ]:
# read the csv file
df = pd.read_csv('fairfaxcountyVA.csv', index_col = 0) # index_col = 0 to remove the index col
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 466 entries, 0 to 465
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Posted      466 non-null    object 
 1   Location    466 non-null    object 
 2   Post Title  466 non-null    object 
 3   Bedrooms    459 non-null    float64
 4   Sqft        265 non-null    float64
 5   Distance    466 non-null    float64
 6   Price       466 non-null    int64  
dtypes: float64(3), int64(1), object(3)
memory usage: 29.1+ KB


# DC

In [ ]:
# Define headers
Headers = ({'User-Agent': 
            'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.114 Safari/537.36'})

# Make a GET request to the initial page to get the total number of pages
response = requests.get("https://www.homes.com/washington-dc/homes-for-rent/2-to-5-bedroom/?property_type=1,2&bath-min=1&bath-max=5", headers=Headers)

if response.status_code == 200:
    first_soup = BeautifulSoup(response.text, 'html.parser')
    # Find the total number of pages
    total_pages = first_soup.find('span', class_='pageRange').text
    total_pages = int(total_pages[-2:])
else:
    print("Failed to retrieve the total number of pages.")
    total_pages = 0

addresses = []
prices = []
bedrooms = []
bathrooms = []
titles = []

# Initiate pages
pages = np.arange(1, total_pages + 1)

i = 0

for page in pages:
    page_str = str(page)
    # Get request
    response = requests.get(f"https://www.homes.com/washington-dc/homes-for-rent/2-to-5-bedroom/p{page_str}/?property_type=1,2&bath-min=1&bath-max=5", headers=Headers)
    
    # Pause the loop
    sleep(random.randint(5, 10))
    
    # Throw a warning for status codes that are not 200
    if response.status_code != 200:
        warnings.warn(f'Request for page {page} failed with status code: {response.status_code}')
        continue  # Skip processing this page and continue with the next
    
    # Define the HTML text
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Define the posts
    posts = soup.find_all('li', class_="placard-container")
    
    # Extract data item-wise
    for post in posts:
        if post.find('ul', class_='detailed-info-container') is not None:
            # Find all <li> elements within the <ul> element
            li_elements = post.find('ul', class_='detailed-info-container').find_all('li')

            # Initialize variables for price, bedroom, and bathroom
            price = "Na"
            bedroom = "Na"
            bathroom = "Na"

            # Process the text from each <li> element
            for li in li_elements:
                text = li.get_text()
                
                # Check if the text contains "per month" (indicating it's a price)
                if "per month" in text:
                    # Check if multiple prices (indicated by "-"), and assign "Na"
                    if "-" in text:
                        price = "Na"
                    else:
                        price = text[:-10]
                        # Otherwise, get only the price by removing "$" and ","
                        price = text.replace('$', '').replace(',', '')
                
                # Check if the text contains "Beds" (indicating it's the number of bedrooms)
                if "Beds" in text:
                    if "-" not in text:
                        bedroom = int(text.split()[0])

                # Check if the text contains "Baths" (indicating it's the number of bathrooms)
                if "Baths" in text:
                    if "-" not in text:
                        bathroom = float(text.split()[0])

            # Append the extracted data to respective lists
            prices.append(price)
            bedrooms.append(bedroom)
            bathrooms.append(bathroom)

        if post.find('p', class_='address') is not None:
            address = post.find('p', class_='address').text
            addresses.append(address)
        if post.find('p', class_='property-name') is not None:
            title = post.find('p', class_='property-name').text
            titles.append(title)
    
    i += 1
    print("Page " + str(i) + " scraped successfully!")

print("\n")
print("Scrape complete!")


Page 1 scraped successfully!
Page 2 scraped successfully!
Page 3 scraped successfully!
Page 4 scraped successfully!
Page 5 scraped successfully!
Page 6 scraped successfully!
Page 7 scraped successfully!
Page 8 scraped successfully!
Page 9 scraped successfully!
Page 10 scraped successfully!
Page 11 scraped successfully!
Page 12 scraped successfully!
Page 13 scraped successfully!
Page 14 scraped successfully!


Scrape complete!


In [ ]:
dc = pd.DataFrame({'Title': titles,
                       'Address': addresses,
                       'Bedroom': bedrooms,
                       'Bathroom': bathrooms,
                        'Price': prices,
                        })
print(dc.info())
dc.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 348 entries, 0 to 347
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Title     348 non-null    object
 1   Address   348 non-null    object
 2   Bedroom   348 non-null    int64 
 3   Bathroom  348 non-null    object
 4   Price     348 non-null    object
dtypes: int64(1), object(4)
memory usage: 13.7+ KB
None


,Title,Address,Bedroom,Bathroom,Price
0,Townhouse for Rent,"747 Kentucky Ave SE, Washington, DC 20003",3,2.0,3400 per month
1,Townhouse for Rent,"5120 Brooks St NE, Washington, DC 20019",3,2.0,2650 per month
2,Townhouse for Rent,"910 New Hampshire Ave NW, Washington, DC 20037",4,3.0,7500 per month
3,Townhouse for Rent,"1224 Irving St NW Unit A, Washington, DC 20010",5,3.0,5975 per month
4,House for Rent,"1112 51st Place NE, Washington, DC 20019",4,3.0,3300 per month
5,Townhouse for Rent,"438 New Jersey Ave SE, Washington, DC 20003",6,4.5,11000 per month
6,Townhouse for Rent,"1818 Corcoran St NE, Washington, DC 20002",2,Na,2100 per month
7,House for Rent,"3906 24th St NE, Washington, DC 20018",3,2.5,3500 per month
8,Townhouse for Rent,"118 17th St SE, Washington, DC 20003",3,3.5,5000 per month
9,Townhouse for Rent,"1326 Otis Place NW, Washington, DC 20010",3,2.5,4500 per month


In [ ]:
#write to csv file for later use
dc.to_csv('dc.csv')

# Arlington County

In [ ]:
# Define headers
Headers = ({'User-Agent': 
            'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.114 Safari/537.36'})

# Make a GET request to the initial page to get the total number of pages
response = requests.get("https://www.homes.com/arlington-county-va/homes-for-rent/2-to-5-bedroom/?property_type=1,2&bath-min=1&bath-max=5", headers=Headers)

if response.status_code == 200:
    first_soup = BeautifulSoup(response.text, 'html.parser')
    # Find the total number of pages
    total_pages = first_soup.find('span', class_='pageRange').text
    total_pages = int(total_pages[-2:])
else:
    print("Failed to retrieve the total number of pages.")
    total_pages = 0

addresses = []
prices = []
bedrooms = []
bathrooms = []
titles = []

# Initiate pages
pages = np.arange(1, total_pages + 1)

i = 0

for page in pages:
    page_str = str(page)
    # Get request
    response = requests.get(f"https://www.homes.com/arlington-county-va/homes-for-rent/2-to-5-bedroom/p{page_str}/?property_type=1,2&bath-min=1&bath-max=5", headers=Headers)
    
    # Pause the loop
    sleep(random.randint(5, 10))
    
    # Throw a warning for status codes that are not 200
    if response.status_code != 200:
        warnings.warn(f'Request for page {page} failed with status code: {response.status_code}')
        continue  # Skip processing this page and continue with the next
    
    # Define the HTML text
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Define the posts
    posts = soup.find_all('li', class_="placard-container")
    
    # Extract data item-wise
    for post in posts:
        if post.find('ul', class_='detailed-info-container') is not None:
            # Find all <li> elements within the <ul> element
            li_elements = post.find('ul', class_='detailed-info-container').find_all('li')

            # Initialize variables for price, bedroom, and bathroom
            price = "Na"
            bedroom = "Na"
            bathroom = "Na"

            # Process the text from each <li> element
            for li in li_elements:
                text = li.get_text()
                
                # Check if the text contains "per month" (indicating it's a price)
                if "per month" in text:
                    # Check if multiple prices (indicated by "-"), and assign "Na"
                    if "-" in text:
                        price = "Na"
                    else:
                        price = text[:-10]
                        # Otherwise, get only the price by removing "$" and ","
                        price = text.replace('$', '').replace(',', '')
                
                # Check if the text contains "Beds" (indicating it's the number of bedrooms)
                if "Beds" in text:
                    if "-" not in text:
                        bedroom = int(text.split()[0])

                # Check if the text contains "Baths" (indicating it's the number of bathrooms)
                if "Baths" in text:
                    if "-" not in text:
                        bathroom = float(text.split()[0])

            # Append the extracted data to respective lists
            prices.append(price)
            bedrooms.append(bedroom)
            bathrooms.append(bathroom)

        if post.find('p', class_='address') is not None:
            address = post.find('p', class_='address').text
            addresses.append(address)
        if post.find('p', class_='property-name') is not None:
            title = post.find('p', class_='property-name').text
            titles.append(title)
    
    i += 1
    print("Page " + str(i) + " scraped successfully!")

print("\n")
print("Scrape complete!")


Page 1 scraped successfully!
Page 2 scraped successfully!
Page 3 scraped successfully!
Page 4 scraped successfully!


Scrape complete!


In [ ]:
arlington = pd.DataFrame({'Title': titles,
                       'Address': addresses,
                       'Bedroom': bedrooms,
                       'Bathroom': bathrooms,
                        'Price': prices,
                        })
print(arlington.info())
arlington.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 86 entries, 0 to 85
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Title     86 non-null     object
 1   Address   86 non-null     object
 2   Bedroom   86 non-null     int64 
 3   Bathroom  86 non-null     object
 4   Price     86 non-null     object
dtypes: int64(1), object(4)
memory usage: 3.5+ KB
None


,Title,Address,Bedroom,Bathroom,Price
0,Townhouse for Rent,"4313 4th Ct N, Arlington, VA 22203",4,4.5,5200 per month
1,House for Rent,"2026 S Monroe St, Arlington, VA 22204",4,3.5,3900 per month
2,House for Rent,"1941 N Vermont St, Arlington, VA 22207",3,2.0,3600 per month
3,Townhouse for Rent,"748 S Granada St, Arlington, VA 22204",4,5.0,3720 per month
4,Townhouse for Rent,"4173 S Four Mile Run Dr Unit B, Arlington, VA ...",3,2.5,3200 per month
5,House for Rent,"1010 19th St S, Arlington, VA 22202",5,3.0,4950 per month
6,Townhouse for Rent,"402 N Thomas St, Arlington, VA 22203",3,2.5,4250 per month
7,House for Rent,"2431 N Nottingham St, Arlington, VA 22207",3,2.5,4200 per month
8,Townhouse for Rent,"2029 S Glebe Rd, Arlington, VA 22204",3,2.0,3399 per month
9,Townhouse for Rent,"2312 N Van Buren Ct, Arlington, VA 22205",3,3.5,4500 per month


In [ ]:
#write to csv file for later use
arlington.to_csv('arlington.csv')

# Prince Williams County

In [30]:
# Define headers
Headers = ({'User-Agent': 
            'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.114 Safari/537.36'})

# Make a GET request to the initial page to get the total number of pages
response = requests.get("https://www.homes.com/prince-william-county-va/homes-for-rent/2-to-5-bedroom/?property_type=1,2&bath-min=1&bath-max=5", headers=Headers)

if response.status_code == 200:
    first_soup = BeautifulSoup(response.text, 'html.parser')
    # Find the total number of pages
    total_pages = first_soup.find('span', class_='pageRange').text
    total_pages = int(total_pages[-2:])
else:
    print("Failed to retrieve the total number of pages.")
    total_pages = 0

addresses = []
prices = []
bedrooms = []
bathrooms = []
titles = []

# Initiate pages
pages = np.arange(1, total_pages + 1)

i = 0

for page in pages:
    page_str = str(page)
    # Get request
    response = requests.get(f"https://www.homes.com/prince-william-county-va/homes-for-rent/2-to-5-bedroom/p{page_str}/?property_type=1,2&bath-min=1&bath-max=5", headers=Headers)
    
    # Pause the loop
    sleep(random.randint(5, 10))
    
    # Throw a warning for status codes that are not 200
    if response.status_code != 200:
        warnings.warn(f'Request for page {page} failed with status code: {response.status_code}')
        continue  # Skip processing this page and continue with the next
    
    # Define the HTML text
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Define the posts
    posts = soup.find_all('li', class_="placard-container")
    
    # Extract data item-wise
    for post in posts:
        if post.find('ul', class_='detailed-info-container') is not None:
            # Find all <li> elements within the <ul> element
            li_elements = post.find('ul', class_='detailed-info-container').find_all('li')

            # Initialize variables for price, bedroom, and bathroom
            price = "Na"
            bedroom = "Na"
            bathroom = "Na"

            # Process the text from each <li> element
            for li in li_elements:
                text = li.get_text()
                
                # Check if the text contains "per month" (indicating it's a price)
                if "per month" in text:
                    # Check if multiple prices (indicated by "-"), and assign "Na"
                    if "-" in text:
                        price = "Na"
                    else:
                        price = text[:-10]
                        # Otherwise, get only the price by removing "$" and ","
                        price = text.replace('$', '').replace(',', '')
                
                # Check if the text contains "Beds" (indicating it's the number of bedrooms)
                if "Beds" in text:
                    if "-" not in text:
                        bedroom = int(text.split()[0])

                # Check if the text contains "Baths" (indicating it's the number of bathrooms)
                if "Baths" in text:
                    if "-" not in text:
                        bathroom = float(text.split()[0])

            # Append the extracted data to respective lists
            prices.append(price)
            bedrooms.append(bedroom)
            bathrooms.append(bathroom)

        if post.find('p', class_='address') is not None:
            address = post.find('p', class_='address').text
            addresses.append(address)
        if post.find('p', class_='property-name') is not None:
            title = post.find('p', class_='property-name').text
            titles.append(title)
    
    i += 1
    print("Page " + str(i) + " scraped successfully!")

print("\n")
print("Scrape complete!")


Page 1 scraped successfully!
Page 2 scraped successfully!
Page 3 scraped successfully!
Page 4 scraped successfully!
Page 5 scraped successfully!
Page 6 scraped successfully!
Page 7 scraped successfully!
Page 8 scraped successfully!
Page 9 scraped successfully!


Scrape complete!


In [32]:
prince_william = pd.DataFrame({'Title': titles,
                       'Address': addresses,
                       'Bedroom': bedrooms,
                       'Bathroom': bathrooms,
                        'Price': prices,
                        })
print(prince_william.info())
prince_william.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 216 entries, 0 to 215
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Title     216 non-null    object
 1   Address   216 non-null    object
 2   Bedroom   216 non-null    int64 
 3   Bathroom  216 non-null    object
 4   Price     216 non-null    object
dtypes: int64(1), object(4)
memory usage: 8.6+ KB
None


,Title,Address,Bedroom,Bathroom,Price
0,House for Rent,"8932 Claret Way, Bristow, VA 20136",5,3.5,3295 per month
1,House for Rent,"10245 Picturesque Ct, Nokesville, VA 20181",4,3.5,3450 per month
2,House for Rent,"1439 Maryland Ave, Woodbridge, VA 22191",3,2.0,2799 per month
3,Townhouse for Rent,"13050 Terminal Way, Woodbridge, VA 22193",4,4.0,2500 per month
4,Townhouse for Rent,"7609 Monitor Ct, Manassas, VA 20109",3,2.5,2395 per month
5,Townhouse for Rent,"5021 Barnacle Place, Dumfries, VA 22025",3,3.5,2650 per month
6,Townhouse for Rent,"12902 Titania Way, Woodbridge, VA 22192",2,2.5,2142 per month
7,House for Rent,"7629 Hamelin Ln, Gainesville, VA 20155",4,2.5,3442 per month
8,Townhouse for Rent,"14807 Ensor Ct, Woodbridge, VA 22193",4,3.0,2200 per month
9,Townhouse for Rent,"16087 Pitner St, Haymarket, VA 20169",3,4.0,2992 per month


In [33]:
#write to csv file for later use
prince_william.to_csv('prince_william.csv')

# Mclean

In [48]:
# Define headers
Headers = ({'User-Agent': 
            'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.114 Safari/537.36'})

# Make a GET request to the initial page to get the total number of pages
response = requests.get("https://www.homes.com/mclean-va/homes-for-rent/2-to-5-bedroom/?property_type=1,2&bath-min=1&bath-max=5", headers=Headers)

if response.status_code == 200:
    first_soup = BeautifulSoup(response.text, 'html.parser')
    # Find the total number of pages
    total_pages = first_soup.find('span', class_='pageRange').text
    total_pages = int(total_pages[-2:])
else:
    print("Failed to retrieve the total number of pages.")
    total_pages = 0

addresses = []
prices = []
bedrooms = []
bathrooms = []
titles = []

# Initiate pages
pages = np.arange(1, total_pages + 1)

i = 0

for page in pages:
    page_str = str(page)
    # Get request
    response = requests.get(f"https://www.homes.com/mclean-va/homes-for-rent/2-to-5-bedroom/p{page_str}/?property_type=1,2&bath-min=1&bath-max=5", headers=Headers)
    
    # Pause the loop
    sleep(random.randint(5, 10))
    
    # Throw a warning for status codes that are not 200
    if response.status_code != 200:
        warnings.warn(f'Request for page {page} failed with status code: {response.status_code}')
        continue  # Skip processing this page and continue with the next
    
    # Define the HTML text
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Define the posts
    posts = soup.find_all('li', class_="placard-container")
    
    # Extract data item-wise
    for post in posts:
        if post.find('ul', class_='detailed-info-container') is not None:
            # Find all <li> elements within the <ul> element
            li_elements = post.find('ul', class_='detailed-info-container').find_all('li')

            # Initialize variables for price, bedroom, and bathroom
            price = "Na"
            bedroom = "Na"
            bathroom = "Na"

            # Process the text from each <li> element
            for li in li_elements:
                text = li.get_text()
                
                # Check if the text contains "per month" (indicating it's a price)
                if "per month" in text:
                    # Check if multiple prices (indicated by "-"), and assign "Na"
                    if "-" in text:
                        price = "Na"
                    else:
                        price = text[:-10]
                        # Otherwise, get only the price by removing "$" and ","
                        price = text.replace('$', '').replace(',', '')
                
                # Check if the text contains "Beds" (indicating it's the number of bedrooms)
                if "Beds" in text:
                    if "-" not in text:
                        bedroom = int(text.split()[0])

                # Check if the text contains "Baths" (indicating it's the number of bathrooms)
                if "Baths" in text:
                    if "-" not in text:
                        bathroom = float(text.split()[0])

            # Append the extracted data to respective lists
            prices.append(price)
            bedrooms.append(bedroom)
            bathrooms.append(bathroom)

        if post.find('p', class_='address') is not None:
            address = post.find('p', class_='address').text
            addresses.append(address)
        if post.find('p', class_='property-name') is not None:
            title = post.find('p', class_='property-name').text
            titles.append(title)
    
    i += 1
    print("Page " + str(i) + " scraped successfully!")

print("\n")
print("Scrape complete!")


Page 1 scraped successfully!
Page 2 scraped successfully!
Page 3 scraped successfully!
Page 4 scraped successfully!


Scrape complete!


In [49]:
mclean = pd.DataFrame({'Title': titles,
                       'Address': addresses,
                       'Bedroom': bedrooms,
                       'Bathroom': bathrooms,
                        'Price': prices,
                        })
print(mclean.info())
mclean.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 97 entries, 0 to 96
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Title     97 non-null     object
 1   Address   97 non-null     object
 2   Bedroom   97 non-null     object
 3   Bathroom  97 non-null     object
 4   Price     97 non-null     object
dtypes: object(5)
memory usage: 3.9+ KB
None


,Title,Address,Bedroom,Bathroom,Price
0,Brentford at The Mile,"7970 Maitland St, McLean, VA 22102",Na,Na,Na
1,Townhouse for Rent,"1708 Anderson Rd, McLean, VA 22102",3,5.0,4990 per month
2,House for Rent,"1348 Macbeth St, McLean, VA 22102",4,4.0,4900 per month
3,House for Rent,"7106 Thrasher Rd, McLean, VA 22101",7,6.5,9800 per month
4,House for Rent,"7008 Hamel Hill Ct, McLean, VA 22101",6,6.5,9600 per month
5,Townhouse for Rent,"7224 Vistas Ln, McLean, VA 22101",3,2.5,4850 per month
6,House for Rent,"1654 Great Falls St, McLean, VA 22101",5,5.0,8500 per month
7,House for Rent,"1556 Hane St, McLean, VA 22101",5,3.0,3850 per month
8,House for Rent,"9428 Meadow Shire Ln, Great Falls, VA 22066",5,3.0,5500 per month
9,House for Rent,"8726 Brook Rd, McLean, VA 22102",7,6.0,8950 per month


In [50]:
#write to csv file for later use
mclean.to_csv('mclean.csv')

# Data Processing & Validation

- Merge all df
- Extract title
- Extract City, State, Zip
- Extract price

## Reload data from csv files

In [3]:
path = '/Users/jenniferma/Desktop/rentals/data/'

In [4]:
# you would not want to run the scraping scripts if you already ran and saved the data. Just upload the files:
fairfax_county_va = pd.read_csv(path + 'fairfaxcountyVA.csv', index_col=[0])  # index col = [0] to ignore the index col in the csv files
arlington = pd.read_csv(path +'arlington.csv', index_col=[0])
dc = pd.read_csv(path + 'dc.csv', index_col=[0])
prince_william = pd.read_csv(path + 'prince_william.csv', index_col=[0])
mclean = pd.read_csv(path + 'mclean.csv', index_col=[0])

In [5]:
# merge all the dataframe in one
rentals = pd.concat([fairfax_county_va, arlington, dc, prince_william, mclean])

In [6]:
rentals.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1251 entries, 0 to 96
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Title     1251 non-null   object
 1   Address   1251 non-null   object
 2   Bedroom   1251 non-null   object
 3   Bathroom  1251 non-null   object
 4   Price     1251 non-null   object
dtypes: object(5)
memory usage: 58.6+ KB


In [7]:
rentals.head()

,Title,Address,Bedroom,Bathroom,Price
0,Brentford at The Mile,"7970 Maitland St, McLean, VA 22102",Na,Na,Na
1,House for Rent,"5929 Merritt Place, Falls Church, VA 22041",5,3.0,4150 per month
2,Townes at Herndon,"508 Pride Ave, Herndon, VA 20170",Na,Na,Na
3,Townhouse for Rent,Unlisted Address,3,3.5,2775 per month
4,House for Rent,"703 Morningside Ct, Herndon, VA 20170",5,2.0,3300 per month


## Data Processing

In [8]:
# drop rows that have Na
rentals = rentals[rentals.Bedroom != 'Na']
rentals.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1248 entries, 1 to 96
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Title     1248 non-null   object
 1   Address   1248 non-null   object
 2   Bedroom   1248 non-null   object
 3   Bathroom  1248 non-null   object
 4   Price     1248 non-null   object
dtypes: object(5)
memory usage: 58.5+ KB


In [9]:
# check again if there is still Na in the dataframe
rentals[rentals.Bedroom == 'Na']

,Title,Address,Bedroom,Bathroom,Price


In [10]:
# extract townhouse or house for titles
# extracting just Townhouse or House
def extract_title(col):
    pattern = r'(Townhouse|House) for Rent'
    match = re.search(pattern, col)
    if match:
        house_type = match.group(1)
        return house_type
    else:
        None

In [11]:
# let's apply to title col
rentals['Title'] = rentals['Title'].apply(extract_title)
rentals.head()

,Title,Address,Bedroom,Bathroom,Price
1,House,"5929 Merritt Place, Falls Church, VA 22041",5,3.0,4150 per month
3,Townhouse,Unlisted Address,3,3.5,2775 per month
4,House,"703 Morningside Ct, Herndon, VA 20170",5,2.0,3300 per month
5,Townhouse,"5455 New London Park Dr, Fairfax, VA 22032",3,4.0,2700 per month
6,Townhouse,"14312 Beckett Glen Cir, Chantilly, VA 20151",3,5.0,4800 per month


In [12]:
# extract state, city, and zipcode
def extract_address(address):
    # Define a regular expression pattern to capture city, state, and zipcode
    pattern = r', (.*), (..) (\d{5})$'

    # Use re.search() to find the pattern in the address
    match = re.search(pattern, address)

    # Check if a match is found and extract the city, state, and zipcode
    if match:
        city = match.group(1).strip()  # Remove leading/trailing spaces
        state = match.group(2)
        zipcode = match.group(3)
        return city, state, zipcode
    else:
        return None, None, None


In [13]:
# apply the function
rentals[['City', 'State', 'Zipcode']] = rentals['Address'].apply(lambda x: pd.Series(extract_address(x)))
rentals.head()

,Title,Address,Bedroom,Bathroom,Price,City,State,Zipcode
1,House,"5929 Merritt Place, Falls Church, VA 22041",5,3.0,4150 per month,Falls Church,VA,22041
3,Townhouse,Unlisted Address,3,3.5,2775 per month,None,None,None
4,House,"703 Morningside Ct, Herndon, VA 20170",5,2.0,3300 per month,Herndon,VA,20170
5,Townhouse,"5455 New London Park Dr, Fairfax, VA 22032",3,4.0,2700 per month,Fairfax,VA,22032
6,Townhouse,"14312 Beckett Glen Cir, Chantilly, VA 20151",3,5.0,4800 per month,Chantilly,VA,20151


In [14]:
# remove "per month"
rentals['Price'] = rentals['Price'].apply(lambda x: x.replace(' per month', '') )
rentals.head()

,Title,Address,Bedroom,Bathroom,Price,City,State,Zipcode
1,House,"5929 Merritt Place, Falls Church, VA 22041",5,3.0,4150,Falls Church,VA,22041
3,Townhouse,Unlisted Address,3,3.5,2775,None,None,None
4,House,"703 Morningside Ct, Herndon, VA 20170",5,2.0,3300,Herndon,VA,20170
5,Townhouse,"5455 New London Park Dr, Fairfax, VA 22032",3,4.0,2700,Fairfax,VA,22032
6,Townhouse,"14312 Beckett Glen Cir, Chantilly, VA 20151",3,5.0,4800,Chantilly,VA,20151


In [15]:
rentals.head()

,Title,Address,Bedroom,Bathroom,Price,City,State,Zipcode
1,House,"5929 Merritt Place, Falls Church, VA 22041",5,3.0,4150,Falls Church,VA,22041
3,Townhouse,Unlisted Address,3,3.5,2775,None,None,None
4,House,"703 Morningside Ct, Herndon, VA 20170",5,2.0,3300,Herndon,VA,20170
5,Townhouse,"5455 New London Park Dr, Fairfax, VA 22032",3,4.0,2700,Fairfax,VA,22032
6,Townhouse,"14312 Beckett Glen Cir, Chantilly, VA 20151",3,5.0,4800,Chantilly,VA,20151


In [16]:
# drop the rows that have None in Zipcode
#rentals = rentals[rentals.Zipcode != 'None']
rentals = rentals[pd.notna(rentals['Zipcode'])]


In [17]:
rentals.head()

,Title,Address,Bedroom,Bathroom,Price,City,State,Zipcode
1,House,"5929 Merritt Place, Falls Church, VA 22041",5,3.0,4150,Falls Church,VA,22041
4,House,"703 Morningside Ct, Herndon, VA 20170",5,2.0,3300,Herndon,VA,20170
5,Townhouse,"5455 New London Park Dr, Fairfax, VA 22032",3,4.0,2700,Fairfax,VA,22032
6,Townhouse,"14312 Beckett Glen Cir, Chantilly, VA 20151",3,5.0,4800,Chantilly,VA,20151
7,House,"12665 Magna Carta Rd, Herndon, VA 20171",3,2.0,2800,Herndon,VA,20171


In [18]:
rentals.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1245 entries, 1 to 96
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Title     1245 non-null   object
 1   Address   1245 non-null   object
 2   Bedroom   1245 non-null   object
 3   Bathroom  1245 non-null   object
 4   Price     1245 non-null   object
 5   City      1245 non-null   object
 6   State     1245 non-null   object
 7   Zipcode   1245 non-null   object
dtypes: object(8)
memory usage: 87.5+ KB


I have to convert Zipcode to str type, not object otherwise I cannot join later on to get the merged_rentals

In [19]:
rentals['Zipcode'] = rentals['Zipcode'].astype('str')

In [20]:
rentals.Zipcode.duplicated()

1     False
4     False
5     False
6     False
7     False
      ...  
92     True
93     True
94     True
95     True
96     True
Name: Zipcode, Length: 1245, dtype: bool

## Data Validation

### Data types
We need to convert some columns to numeric types

In [21]:
# convert to numeric
cols = ['Bedroom', 'Bathroom','Price']
rentals[cols] = rentals[cols].apply(pd.to_numeric, errors='coerce')

In [22]:
rentals.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1245 entries, 1 to 96
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Title     1245 non-null   object 
 1   Address   1245 non-null   object 
 2   Bedroom   1245 non-null   int64  
 3   Bathroom  1166 non-null   float64
 4   Price     1245 non-null   int64  
 5   City      1245 non-null   object 
 6   State     1245 non-null   object 
 7   Zipcode   1245 non-null   object 
dtypes: float64(1), int64(2), object(5)
memory usage: 87.5+ KB


In [23]:
rentals.describe()

,Bedroom,Bathroom,Price
count,1245.000000,1166.000000,1245.000000
mean,3.536546,3.239708,4200.319679
std,1.143831,1.235821,2963.026909
min,2.000000,1.500000,1000.000000
25%,3.000000,2.500000,2800.000000
50%,3.000000,3.000000,3400.000000
75%,4.000000,3.500000,4500.000000
max,12.000000,13.000000,40000.000000


### Outliers

There is a house that has 12 bedrooms

In [24]:
rentals[rentals['Bedroom'] == 12]

,Title,Address,Bedroom,Bathroom,Price,City,State,Zipcode
117,House,"17571 Graham St, Dumfries, VA 22026",12,6.0,9150,Dumfries,VA,22026


This house has 13 bathrooms, should be a typo. Let's drop it 

In [25]:
rentals[rentals['Bathroom'] == 13]

,Title,Address,Bedroom,Bathroom,Price,City,State,Zipcode
103,Townhouse,"13172 Putnam Cir, Woodbridge, VA 22191",3,13.0,2500,Woodbridge,VA,22191


In [26]:
# dropping the outlier which has 13 bedrooms
rentals = rentals[rentals['Bathroom'] != 13]

### Duplicates

In [27]:
# checking any duplicates
rentals[rentals.duplicated(subset='Address')]

,Title,Address,Bedroom,Bathroom,Price,City,State,Zipcode
287,Townhouse,"1524 16th Ct N, Arlington, VA 22209",3,3.5,6900,Arlington,VA,22209
298,Townhouse,"4120 41st St N Unit 1, Arlington, VA 22207",2,2.0,5700,Arlington,VA,22207
301,Townhouse,"2404 13th Ct N, Arlington, VA 22201",3,3.5,4995,Arlington,VA,22201
302,Townhouse,"2422 14th St N, Arlington, VA 22201",3,3.5,4900,Arlington,VA,22201
305,Townhouse,"606 15th St S Unit 1, Arlington, VA 22202",3,2.5,3295,Arlington,VA,22202
...,...,...,...,...,...,...,...,...
86,House,"8811 Higdon Dr, Vienna, VA 22182",4,2.5,3195,Vienna,VA,22182
87,Townhouse,"2528 N Winchester St, Arlington, VA 22213",2,1.5,2825,Arlington,VA,22213
88,House,"2219 N Nottingham St, Arlington, VA 22205",4,2.0,4100,Arlington,VA,22205
91,House,"1511 Gingerwood Ct, Vienna, VA 22182",4,4.0,3999,Vienna,VA,22182


In [28]:
rentals[rentals.Address == '1524 16th Ct N, Arlington, VA 22209']

,Title,Address,Bedroom,Bathroom,Price,City,State,Zipcode
14,Townhouse,"1524 16th Ct N, Arlington, VA 22209",3,3.5,6900,Arlington,VA,22209
287,Townhouse,"1524 16th Ct N, Arlington, VA 22209",3,3.5,6900,Arlington,VA,22209


In [29]:
rentals[rentals.Address == '1159 Mill Garden Ct, Great Falls, VA 22066']

,Title,Address,Bedroom,Bathroom,Price,City,State,Zipcode
345,House,"1159 Mill Garden Ct, Great Falls, VA 22066",6,6.5,8000,Great Falls,VA,22066
93,House,"1159 Mill Garden Ct, Great Falls, VA 22066",6,6.5,8000,Great Falls,VA,22066


There are 70 duplicates in the rentals df. So, we should drop them to have a more accurate statistical median prices.

In [30]:
rentals.shape

(1244, 8)

In [31]:
rentals = rentals.drop_duplicates()

In [32]:
print(rentals.shape)
print(rentals.duplicated().sum())

(1189, 8)
0


When you just drop duplicates, not specifically on Address, you would miss some duplicates

In [33]:
rentals = rentals.drop_duplicates(subset='Address')

In [34]:
print(rentals.shape)
print(rentals.duplicated(subset='Address').sum())

(1174, 8)
0


## Pandas Profilings

In [44]:
#!pip install ydata-profiling

In [46]:
#!pip freeze

In [48]:
#from pandas_profiling import ProfileReport
#prof = ProfileReport(rentals)
#prof.to_file(output_file='rentals.html')

In [ ]:
#!pip install ipykernel

In [ ]:
#!pip install --upgrade nbformat==4.2.0

## EDA

In [35]:
rentals['Bedroom'].value_counts()

Bedroom
3     494
4     312
2     183
5     135
6      33
7      11
8       4
9       1
12      1
Name: count, dtype: int64

In [26]:
rentals['Bathroom'].value_counts()

Bathroom
3.5     294
2.5     226
2.0     189
3.0     128
4.0     107
4.5      79
1.5      54
5.0      34
6.0      17
5.5      14
6.5       7
7.0       7
9.0       5
8.5       2
9.5       2
10.0      2
8.0       1
Name: count, dtype: int64

### Bedroom Distribution

Anatomy of Plotly Express

import plotly.express as px

fig = px.plotting_function(  
    dataframe,  
    x='column-for-xaxis',  
    y='column-for-yaxis',  
    title='Title For the Plot',  
    width=width_in_pixels,  
    height=height_in_pixels  
)  
  
fig.show()

Most of the houses have 3,4 bedrooms

In [36]:
# trying with plotly express
fig = px.histogram(rentals, x="Bedroom", title="Histogram of Bedrooms",width=600,
  height=400,)
fig.show()

In [86]:
# violin and boxplots
fig = px.violin(rentals, y="Bedroom", box=True, # draw box plot inside the violin
                points='all', # can be 'outliers', or False
                title="Violin Plot of Bedrooms", width=600, height=400,)
fig.show()

Most of them have 2.5 or 3.5 baths

In [37]:
fig = px.histogram(rentals, x="Bathroom", title="Distribution of Bathrooms", width=600,
  height=400,)
fig.show()

In [88]:
# violin and boxplots
fig = px.violin(rentals, y="Bathroom", box=True, # draw box plot inside the violin
                points='all', # can be 'outliers', or False
                width=600, title="Violin Plot of Bathrooms",
                height=400,
               )
fig.show()

Let's check the range of br, ba, price when grouping

In [38]:
grouped_bedroom = rentals.groupby('Bedroom').agg({'Price': ['min', 'max', 'mean']})
#Rename the mutltiindex columns and reset the index
grouped_bedroom.columns = ['price_min', 'price_max', 'price_mean']
grouped_bedroom.reset_index()
grouped_bedroom

,price_min,price_max,price_mean
Bedroom,,,
2,1200,10000,2859.000000
3,1050,20000,3384.578947
4,1000,16500,4174.801282
5,2800,18000,5717.874074
6,3800,13750,7606.969697
7,4500,40000,13522.727273
8,5095,38500,16348.750000
9,19800,19800,19800.000000
12,9150,9150,9150.000000


It looks like we have a big range of price by bedroom. However, I am only interested in a house/townhouse that has 3-4br and at least 2 ba. Let's see how much the median price is?


### Filtering 3,4br and at least 2 ba

In [90]:
# subset 3-4 Bedrooms and at least 2 ba
df_34br_atleast2ba = rentals[((rentals.Bedroom == 3)|(rentals.Bedroom == 4))&(rentals.Bathroom >=2)]
df_34br_atleast2ba.describe()

,Bedroom,Bathroom,Price
count,757.000000,757.000000,757.000000
mean,3.408190,3.104359,3770.464993
std,0.491824,0.792037,1633.885698
min,3.000000,2.000000,1050.000000
25%,3.000000,2.500000,2850.000000
50%,3.000000,3.000000,3375.000000
75%,4.000000,3.500000,4100.000000
max,4.000000,7.000000,20000.000000


In [91]:
# check how many houses which have 7 ba? It looks legit
df_34br_atleast2ba[df_34br_atleast2ba.Bathroom == 7]

,Title,Address,Bedroom,Bathroom,Price,City,State,Zipcode
78,House,"7004 Nevis Rd, Bethesda, MD 20817",4,7.0,6500,Bethesda,MD,20817


I want to see the distribution of houses by bedroom, city?

In [92]:
fig = px.violin(df_34br_atleast2ba, y="Price", x="Bedroom", color="Bedroom", box=True, points="all", hover_data=df_34br_atleast2ba.columns)
fig.show()

There are some outliers above 10k price, most of the houses are under 5k. Probably, the above 10k houses are located in the expensive neighborbood. Let's check and see

In [93]:
expensive_houses = df_34br_atleast2ba[df_34br_atleast2ba.Price >=10000]

In [94]:
expensive_houses

,Title,Address,Bedroom,Bathroom,Price,City,State,Zipcode
53,Townhouse,"1421 N Nash St, Arlington, VA 22209",3,6.0,12000,Arlington,VA,22209
82,Townhouse,"322 Adolf Cluss Ct SE, Washington, DC 20003",3,3.5,12000,Washington,DC,20003
106,House,"3500 P St NW, Washington, DC 20007",4,3.5,10500,Washington,DC,20007
107,Townhouse,"1227 33rd St NW, Washington, DC 20007",4,2.5,10000,Washington,DC,20007
155,Townhouse,"2022 Hillyer Place NW, Washington, DC 20009",4,3.0,15000,Washington,DC,20009
181,Townhouse,"1420 Paloma Way NW, Washington, DC 20009",3,2.5,12500,Washington,DC,20009
184,Townhouse,"1604 13th St NW, Washington, DC 20009",4,3.0,16500,Washington,DC,20009
205,House,"45 Sutton Square SW Unit 908, Washington, DC 2...",3,3.5,20000,Washington,DC,20024
218,House,"1414 35th St NW, Washington, DC 20007",4,2.0,10995,Washington,DC,20007


In [95]:
df_34br_atleast2ba.City.value_counts()

City
Washington           143
Woodbridge            93
Alexandria            62
Vienna                47
Arlington             47
Fairfax               45
Herndon               43
Falls Church          29
Springfield           28
McLean                24
Reston                23
Manassas              21
Dumfries              20
Centreville           19
Burke                 12
Annandale             11
Lorton                 8
Haymarket              8
Gainesville            7
Bethesda               7
Chevy Chase            6
Great Falls            6
Chantilly              5
Bristow                5
Temple Hills           4
Hyattsville            4
Capitol Heights        3
Triangle               3
Silver Spring          3
Oakton                 3
Potomac                2
Mount Rainier          2
Nokesville             2
Sterling               2
Oak Hill               2
Brentwood              1
Cheverly               1
Oxon Hill              1
Fairfax Station        1
Catharpin           

## Problem Statement

In [ ]:
# always update plotly as it receives regular updates
!pip install --upgrade plotly

We are searching for rental houses in the Washington, D.C. area that meet specific criteria to ensure convenience and safety. Our primary considerations include:  

- Metro Proximity: We prioritize houses located near metro stations for easy commuting.  

- Low Crime Index: Safety is paramount, so we are looking for areas with a low crime index.  

- Property Specifications: We require houses with a minimum of 3-4 bedrooms and at least 2 bathrooms.  

To make an informed decision, we plan to compare properties based on these criteria, including metro proximity, crime index, bedroom and bathroom count, city, and zipcode. We aim to calculate the median rental prices for each criterion to help us identify the most suitable options

Save the df to csv file

In [138]:
df_34br_atleast2ba.info()

<class 'pandas.core.frame.DataFrame'>
Index: 757 entries, 5 to 95
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Title     757 non-null    object 
 1   Address   757 non-null    object 
 2   Bedroom   757 non-null    int64  
 3   Bathroom  757 non-null    float64
 4   Price     757 non-null    int64  
 5   City      757 non-null    object 
 6   State     757 non-null    object 
 7   Zipcode   757 non-null    object 
dtypes: float64(1), int64(2), object(5)
memory usage: 53.2+ KB


Adding longitude and latitude

In [142]:
long_lat_rentals = pd.read_excel(path+'long_lat_rentals.xlsx')
long_lat_rentals.head()

,Zipcode,Latitude,Longitude
0,22315,38.7569,-77.1480
1,22314,38.8074,-77.0567
2,22312,38.8177,-77.1538
3,22311,38.8340,-77.1215
4,22310,38.7832,-77.1230


In [143]:
long_lat_rentals['Zipcode'] = long_lat_rentals['Zipcode'].astype('str')

Merging longitude and lattitude into the df_34br_atleast2ba

In [147]:
merged_rentals = df_34br_atleast2ba.merge(long_lat_rentals, on='Zipcode', how='inner')

In [151]:
!pwd

/Users/jenniferma/Desktop/rentals


In [148]:
merged_rentals.to_csv('df_34br_atleast2ba.csv')

for some reasons, after reading the file, Zipcode column again change to int type!!

# LOAD DF34BR_ATLEAST2BA DF

In [40]:
df_34br_atleast2ba = pd.read_csv('df_34br_atleast2ba.csv', index_col=[0])

In [99]:
# let's plot bar chart of top 20 cities with most data 
# Count the occurrences of each city in the DataFrame
city_counts = df_34br_atleast2ba['City'].value_counts()

# Select the top 20 cities counts
top_20_cities = city_counts.head(20).index.tolist()

# Filter the DataFrame to include only rows for the top 20 cities
df_top_20_cities = df_34br_atleast2ba[df_34br_atleast2ba['City'].isin(top_20_cities)]


In [100]:
# get median prices of top10 and bottom10
median_top20cities = df_top_20_cities.groupby('City')['Price'].median().reset_index().sort_values(by='Price')

# plot the top10
fig = px.bar(median_top20cities, y='City', x='Price', orientation='h')
fig.update_layout(
    title="Median Price by City of Top 20 Highest Counts in 2023",
    yaxis_title="",
    xaxis_title="Median Price ($)"
)

fig.show()

In [83]:
# sort cities to show cities in descending order (Price)
sorted_cities = median_top20cities.sort_values(by='Price', ascending=False)
# create a dict to insert in the violin plotly
city_order_dict = {"City": sorted_cities["City"].tolist()}

# Specify the order of cities in the violin plot using category_orders
fig = px.violin(df_top_20_cities, 
                y='City', 
                x='Price', 
                box=True, 
                points='all', 
                orientation='h', 
                hover_data=median_top20cities.columns, 
                height=1000,
                category_orders=city_order_dict
                )
fig.update_layout(title= "Price Distribution of Top 20 Most Counted Cities in 2023 - Rentals",
                  yaxis_title = "",
                  xaxis_title = "Price"
                  )

# Show the plot
fig.show()


Washington DC has the most spread of data. Let's investigate more what cause the differences?

In [125]:
df_top_20_cities[df_top_20_cities.City == 'Washington']

,Title,Address,Bedroom,Bathroom,Price,City,State,Zipcode
0,Townhouse,"747 Kentucky Ave SE, Washington, DC 20003",3,2.0,3400,Washington,DC,20003
1,Townhouse,"5120 Brooks St NE, Washington, DC 20019",3,2.0,2650,Washington,DC,20019
2,Townhouse,"910 New Hampshire Ave NW, Washington, DC 20037",4,3.0,7500,Washington,DC,20037
4,House,"1112 51st Place NE, Washington, DC 20019",4,3.0,3300,Washington,DC,20019
7,House,"3906 24th St NE, Washington, DC 20018",3,2.5,3500,Washington,DC,20018
...,...,...,...,...,...,...,...,...
265,Townhouse,"1230 Eton Ct NW Unit T26, Washington, DC 20007",3,2.5,5500,Washington,DC,20007
266,House,"1500 Ogden St NW, Washington, DC 20010",3,3.0,3600,Washington,DC,20010
268,Townhouse,"3521 Holmead Place NW, Washington, DC 20010",4,2.5,5400,Washington,DC,20010
269,House,"3712 Windom Place NW, Washington, DC 20016",4,4.5,6500,Washington,DC,20016


### Checking correlations

It is interesting to see Bathrooms are more positive correlated to Price than Bedrooms

In [65]:
rentals[['Bedroom', 'Bathroom', 'Price']].corr()

,Bedroom,Bathroom,Price
Bedroom,1.000000,0.637696,0.558082
Bathroom,0.637696,1.000000,0.617843
Price,0.558082,0.617843,1.000000


# Craiglist Rentals in 2021

In [73]:
rentals_21 = pd.read_csv(path+'craiglists.csv', index_col=[0])
rentals_21.head()

,Posted,Location,Post Title,Bedrooms,Sqft,Distance,Price,Clean_location
0,2021-07-17,"fairfax county, annandale",Fenced yard-all utilities included-large 1 bed...,1.0,1000.0,0.0,1290,Fairfax
2,2021-07-14,"Annadale, VA",5B3BA Fully Remodeled Single Family House for ...,5.0,2500.0,1.1,3600,Annandale
4,2021-07-17,North Springfield,"*Beautiful Home, Spacious + Bright 3 Bedroom 2...",3.0,2500.0,1.9,3100,Springfield
5,2021-07-12,Fairfax,"Guest Suite, Designer Fixtures, Contemporary F...",2.0,1216.0,2.3,2015,Fairfax
6,2021-07-14,Fairfax,"Pets Welcome, Lease Today, Perfect Location",2.0,1216.0,2.3,1965,Fairfax


In [75]:
rentals_21 = rentals_21[['Clean_location', 'Bedrooms', 'Price']]
rentals_21.head()

,Clean_location,Bedrooms,Price
0,Fairfax,1.0,1290
2,Annandale,5.0,3600
4,Springfield,3.0,3100
5,Fairfax,2.0,2015
6,Fairfax,2.0,1965


In [76]:
# subset 3-4br of the rentals in 2021 that was scraped from craiglists
rentals_21_34br = rentals_21[(rentals_21.Bedrooms == 3) | (rentals_21.Bedrooms == 4)]
rentals_21_34br.Bedrooms.value_counts()

Bedrooms
3.0    102
4.0     63
Name: count, dtype: int64

In [78]:
# Visualize the distribution of the df
px.histogram(rentals_21_34br, x='Price')

Let's check median prices and their distribution of the rentals by Cities

In [85]:
# let's plot bar chart of top 20 cities with most data 
# Count the occurrences of each city in the DataFrame
city_counts_21 = rentals_21_34br['Clean_location'].value_counts()

# Select the top 20 cities counts
top_20_cities_21 = city_counts_21.head(20).index.tolist()

# Filter the DataFrame to include only rows for the top 20 cities
df_top_20_cities_21 = rentals_21_34br[rentals_21_34br['Clean_location'].isin(top_20_cities_21)]

In [87]:
# get median prices of top10 and bottom10
median_top20cities_21 = df_top_20_cities_21.groupby('Clean_location')['Price'].median().reset_index().sort_values(by='Price')

# plot the top10
fig = px.bar(median_top20cities_21, y='Clean_location', x='Price', orientation='h')
fig.update_layout(
    title="Median Price by City of Top 20 Highest Counts in 2021",
    yaxis_title="",
    xaxis_title="Median Price ($)"
)

fig.show()

In [89]:
# sort cities to show cities in descending order (Price)
sorted_cities = median_top20cities_21.sort_values(by='Price', ascending=False)
# create a dict to insert in the violin plotly
city_order_dict_21 = {"Clean_location": sorted_cities["Clean_location"].tolist()}

# Specify the order of cities in the violin plot using category_orders
fig = px.violin(df_top_20_cities_21, 
                y='Clean_location', 
                x='Price', 
                box=True, 
                points='all', 
                orientation='h', 
                hover_data=median_top20cities_21.columns, 
                height=1000,
                category_orders=city_order_dict
                )
fig.update_layout(title= "Price Distribution of Top 20 Most Counted Cities in 2021 - Rentals",
                  yaxis_title = "",
                  xaxis_title = "Price"
                  )

# Show the plot
fig.show()


In the dataset of craiglist in 2021, we have more data in DC(Washington), Arlington, Alexandria, ...

In [90]:
# how many data points we have for cities
df_top_20_cities_21.Clean_location.value_counts()

Clean_location
DC              75
Arlington       17
Herndon         11
Falls Church    10
Alexandria      10
Fairfax          8
Centreville      5
Woodbridge       5
Springfield      4
Vienna           4
Mclean           3
Lorton           3
Burke            2
Chevy Chase      2
Chantilly        1
Potomac          1
Mount Vernon     1
Reston           1
Oxon Hill        1
Temple Hills     1
Name: count, dtype: int64

In [93]:
import pprint

In [102]:
median_top20cities.City.value_counts()

City
Woodbridge      1
Manassas        1
McLean          1
Washington      1
Arlington       1
Falls Church    1
Vienna          1
Alexandria      1
Annandale       1
Fairfax         1
Dumfries        1
Lorton          1
Gainesville     1
Springfield     1
Herndon         1
Burke           1
Reston          1
Haymarket       1
Centreville     1
Bethesda        1
Name: count, dtype: int64

In [95]:
# let's compare 10 cities of 2021 vs 2023 as I am more interested in the cities. 2023 dataset does not have
# Fairfax city. 
interested_cities = df_top_20_cities_21['Clean_location'].value_counts().index[:11].to_list()
print(interested_cities)

['DC', 'Arlington', 'Herndon', 'Falls Church', 'Alexandria', 'Fairfax', 'Centreville', 'Woodbridge', 'Springfield', 'Vienna', 'Mclean']


In [103]:
interested_cities = ['Washington', 'Arlington', 'Herndon', 'Falls Church', 'Alexandria', 'Centreville', 'Woodbridge', 'Springfield', 'Vienna', 'Mclean']

# Conclusion

It is quite challenging to webscrape in these days as many websites of big companies have all the smart bot detection system to block bots' crawling on their site. To prevent from being detected and then blocked from the sites, we should carefully draft or plan the codes first before starting to scrape data, and try to use headers, proxies, time delays as much as we could.

Craigslist's data is very messy with different formats of locations, bedrooms, bathrooms and prices. It takes a lot of time to clean up the data before data analysis. There is somewhat a correlation between Bedrooms and Price, 0.605. There are mostly 3-4 bedroom houses/townhomes for rent in the market in the Northern Virginia and most of the rentals available are from DC and Arlington VA. Average rental costs for 2, 3, and 4 bedrooms are 2539, 2759, and 3720 per month respectively in general. Specifically, in Arlington average rental costs for 3 and 4 bedroom houses/townhomes are 3382 and 4061 respectively. 

From data, we could see prices in different locations vary significantly. Especially, prices in Arlington are higher compared to ones in Woodbridge. It makes senses since Arlington is close to Washington DC where most jobs in the area are from DC. Workers who live in Arlington could avoid getting hit by traffic and save time of commuting than those who live in Woodbridge, so it causes the prices higher. 
